In [1]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.4/161.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata
gemini_api_key = userdata.get("GOOGLE_API_KEY")

## Global

In [4]:
from agents import AsyncOpenAI, set_default_openai_client, set_tracing_disabled, set_default_openai_api

set_tracing_disabled(True)
set_default_openai_api("chat_completions")

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
set_default_openai_client(external_client)

##Handoff Concept

In [5]:
from agents import Agent, Runner
import asyncio

spenish_agent = Agent(name="spenish_agent", instructions='You are a helpful assistant, who guide user whenver they wat to talk in spenish language.', model = 'gemini-2.5-flash')

urdu_agent = Agent(name="urdu_agent", instructions='You are a helpful assistant, who guide user whenver they wat to talk in urdu language.', model = 'gemini-2.5-flash')

triage_agent = Agent(name = "Assistant", instructions="Be a helpful assistant. If the user wants to speaks Spanish, handoff to the Spanish assistant. But If the user wants to Urdu Spanish, handoff to the Urdu assistant. ", model = 'gemini-2.5-flash', handoffs = [spenish_agent, urdu_agent])


###Asyncronous Function

In [6]:
from agents import Runner

async def main():

  result = await Runner.run(
      triage_agent,
      input= 'Convert "I m a good girl" in Urdu'
  )

  print(f"Response: {result.final_output}")

if __name__ == "__main__":
  asyncio.run(main())

Response: میں ایک اچھی لڑکی ہوں (Main aik achi larki hoon).


##Streaming Function (Validation (logprobs) Error)

In [7]:
from agents import Runner

async def main():

  result = Runner.run_streamed(
      triage_agent,
      input= 'Convert "I m a good girl" in Urdu'
  )

  async for event in result.stream_events():
    print(f"Event: {event.type}")

if __name__ == "__main__":
  asyncio.run(main())

Event: agent_updated_stream_event
Event: raw_response_event
Event: raw_response_event
Event: raw_response_event
Event: raw_response_event
Event: raw_response_event
Event: run_item_stream_event
Event: run_item_stream_event
Event: agent_updated_stream_event
Event: raw_response_event


ValidationError: 1 validation error for ResponseTextDeltaEvent
logprobs
  Field required [type=missing, input_value={'content_index': 0, 'del...', 'sequence_number': 3}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

##DEBUG

In [ ]:
from agents import Runner
from agents import enable_verbose_stdout_logging

enable_verbose_stdout_logging()

async def main():

  result = await Runner.run(
      triage_agent,
      input= 'Convert "I m a good girl" in Urdu'
  )

  print(f"Response: {result.final_output}")

if __name__ == "__main__":
  asyncio.run(main())


Tracing is disabled. Not creating trace Agent workflow


DEBUG:openai.agents:Tracing is disabled. Not creating trace Agent workflow


Setting current trace: no-op


DEBUG:openai.agents:Setting current trace: no-op


Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x7a62026986b0>


...

LLM resp:
{
  "content": "\"I'm a good girl\" in Urdu is:\n\n**میں ایک اچھی لڑکی ہوں**\n(Main ek achi larki hoon)",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



DEBUG:openai.agents:LLM resp:
{
  "content": "\"I'm a good girl\" in Urdu is:\n\n**میں ایک اچھی لڑکی ہوں**\n(Main ek achi larki hoon)",
  "refusal": null,
  "role": "assistant",
  "annotations": null,
  "audio": null,
  "function_call": null,
  "tool_calls": null
}



Resetting current trace


DEBUG:openai.agents:Resetting current trace


Response: "I'm a good girl" in Urdu is:

**میں ایک اچھی لڑکی ہوں**
(Main ek achi larki hoon)
